# Bibliotecas

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
from obterDados import obterSimbolo

# Obtendo dados

In [ ]:
rawDataM5 = obterSimbolo('WDO$')
rawDataM5

# Tratamento de Dados

### Gerando colunas com preços passados e futuros

In [ ]:
hist = rawDataM5.copy().drop(columns=['tick_volume', 'real_volume', 'spread'])
for i in range(1000):
    hist['open'+str(i+1)] = hist['open'].shift(i+1)
    hist['high'+str(i+1)] = hist['high'].shift(i+1)
    hist['low'+str(i+1)] = hist['low'].shift(i+1)
    hist['close'+str(i+1)] = hist['close'].shift(i+1)
colunas0 = ['open', 'high', 'low', 'close']
colunasY = []
for i in range(24):
    for name in colunas0:
        nomeCol = f'{name}-{str(i+1)}'
        hist[nomeCol] = hist[name].shift(-1-i)
        colunasY.append(nomeCol)

hist = hist.dropna()
hist

# Modelo de Classificação

### Normalização

In [ ]:
maxVs = hist.drop(columns=colunasY).max(axis='columns')
minVs = hist.drop(columns=colunasY).min(axis='columns')
histNorm = hist.subtract(minVs, axis=0).divide(maxVs - minVs, axis=0)
histNorm

### Separar dados de treino e teste

In [6]:
xClass = histNorm.drop(columns=colunasY)
yClass = histNorm[colunasY]
splitDays = 108 * 150
x_train_class = xClass.iloc[:-splitDays]
x_test_class = xClass.iloc[-splitDays:]
y_train_class = yClass.iloc[:-splitDays]
y_test_class = yClass.iloc[-splitDays:]

### Lazy Regressor

In [ ]:
from lazypredict.Supervised import LazyClassifier

lazyReg = LazyClassifier()
lazyReg.fit(x_train_class, x_test_class, y_train_class, y_test_class)

### Criando um dos modelos encontrados

In [ ]:
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.neural_network import MLPRegressor
from math import sqrt

# modelo = OrthogonalMatchingPursuit()
modelo = MLPRegressor(verbose=True)
modelo.fit(x_train_class, y_train_class)
y_train_predict = modelo.predict(x_train_class)
y_test_predict = modelo.predict(x_test_class)
print('Treino R2:', r2_score(y_train_class, y_train_predict))
print('Teste R2:', r2_score(y_test_class, y_test_predict))
print('Treino RMSE:', sqrt(mean_squared_error(y_train_class, modelo.predict(x_train_class))))
print('Teste RMSE:', sqrt(mean_squared_error(y_test_class, modelo.predict(x_test_class))))


### Salvar Modelo de Classificação

In [ ]:
import pickle

pickle.dump(modelo, open('modeloClassificador.pickle', 'wb'))